In [1]:
pip install tensorflow tqdm pillow scipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import shutil 
import random
import logging
import scipy
import numpy as np
from tqdm import tqdm
from PIL import Image
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


# **Analyze The Dataset :**

In [4]:
def analyze_dataset(dir):
    
    cls_counts={}
    img_ext=('.jpg', '.jpeg', '.png')
    
    if not os.path.exists(dir):
        print(f"Error!!-Directory [{dir}] can not found.")
        return None
    
    for cls_name in os.listdir(dir):
        cls_path = os.path.join(dir, cls_name)    
        
        if os.path.isdir(cls_path):
            try:
                img_files=[f for f in os.listdir(cls_path) if f.lower().endswith(img_ext)]
                cls_counts[cls_name]=len(img_files)
            except Exception as e:
                print(f"Error occurred during get the fils from the directory [{dir}]: {e}")
                cls_counts[cls_name]=0
                
    total_imgs=sum(cls_counts.values())
    print(f"Total Images : {total_imgs}")
    
    for cls_name, count in sorted(cls_counts.items()):
        percentage =(count/total_imgs*100) if total_imgs>0 else 0
        print(f"[{cls_name}] : {count} Images, [{percentage:.1f}%]")
    
    return cls_counts

In [5]:
analyze_dataset(dir="F:/MobileNetV2/Dataset/Skin diseases data")

Total Images : 755
[Chickenpox] : 75 Images, [9.9%]
[Cowpox] : 66 Images, [8.7%]
[HFMD] : 161 Images, [21.3%]
[Healthy] : 114 Images, [15.1%]
[Measles] : 55 Images, [7.3%]
[Monkeypox] : 284 Images, [37.6%]


{'Chickenpox': 75,
 'Cowpox': 66,
 'Healthy': 114,
 'HFMD': 161,
 'Measles': 55,
 'Monkeypox': 284}

# **Balance The Classes Using Data Augmentation :**

In [7]:
def augmentation(input_dir, output_dir, target_images, save_format, random_seed):
   
    if not os.path.exists(input_dir):
        print(f"Error!!-Directory [{input_dir}] cannot be found.")
        return None    
    
    # For reproducibility
    np.random.seed(random_seed)
    img_ext = ('.jpg', '.jpeg', '.png')
    
    # Create data generator 
    data_gen= ImageDataGenerator(rotation_range=5,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 zoom_range=0.1,
                                 brightness_range=[0.95, 1.05],
                                 width_shift_range=0.05,
                                 height_shift_range=0.05,
                                 fill_mode='nearest' # fill new pixels with nearest value                                   
    )
    
    try:
        img_files = [f for f in os.listdir(input_dir) if f.lower().endswith(img_ext)]
        current_images = len(img_files)
        print(f"Found [{current_images}] images in [{input_dir}] path.")
    except Exception as e:
        print(f"Error occurred during getting the fils from the directory [{input_dir}]: {e}")
        return None
    
    # Check if any images were found 
    if current_images == 0:
        print(f"No images found in directory [{input_dir}].")
        return None
    
    # Check augmentation is needed or not 
    if current_images >= target_images:
        print(f"Warning!!-Current images [{current_images}] >= target [{target_images}], So no augmentation needed.")
        return 0
    
    # Create output directory 
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Output directory created or verified : [{output_dir}].")
    except Exception as e:
        print(f"Error occurred during create the directory [{output_dir}]: {e}")
        return None
        
    # Calculate images to generate
    imgs_to_gen = target_images - current_images
    print(f"Need to generate [{imgs_to_gen}] additional images to balance the dataset.")
    
    # Calculate distribution of augmented images per original image 
    imgs_per_original = imgs_to_gen // current_images 
    remainder = imgs_to_gen % current_images
    
    print(f"Generating [{imgs_per_original}] images per original, with [{remainder}] images getting one extra.")
    total_generated = 0
    
    # Process each image from directory
    for idx, img_name in enumerate(tqdm(img_files, desc="Augmenting images")):
        try:
            img_path = os.path.join(input_dir, img_name)
            
            # load and preprocess image
            img=load_img(img_path)
            img_array=img_to_array(img)
            img_array=img_array.reshape((1,) + img_array.shape)
            
            # calculate how many augmented images to generate for this original
            target_per_img = imgs_per_original + (1 if idx < remainder else 0)
            
            # skip if no images needed for this original
            if target_per_img == 0:
                continue
            
            #Generate augmented images
            generated_count = 0
            base_name=os.path.splitext(img_name)[0]
            
            # Create a generator and iterate through it
            aug_iter = data_gen.flow(
                img_array,batch_size=1,
                save_to_dir=output_dir,
                save_prefix=f'aug_{base_name}_',
                save_format=save_format
            ) 

            # Generate the required number of augmented images 
            for i in range(target_per_img):
                try:
                    next(aug_iter) # Generate one augmented image
                    generated_count += 1
                    total_generated += 1
                
                except StopIteration:
                    print(f"Warning!!-Generator exhausted for [{img_name}] after [{generated_count}] images.")
                    break
                
        except Exception as e:
            print(f"Error occurred during processing [{img_name}]: {str(e)}")
            continue
        
    print(f"Augmentation complete! Generated [{total_generated}] new images.")
    print(f"Total images now [{current_images + total_generated}] new images.")  
    
    return total_generated  

In [8]:
# Augmentation multiple classes together 
def multi_aug_classes(input_dir, output_dir, class_targets, save_format, random_seed):
    
    for class_name, target_count in class_targets.items():
        input_path = os.path.join(input_dir, class_name)
        output_path = os.path.join(output_dir, class_name)
        
        print(f"Processing class: [{class_name}]")
        
        try:
            augmentation(input_dir = input_path,
                         output_dir = output_path,
                         target_images = target_count,
                         save_format = save_format,
                         random_seed = random_seed)
            
            # Verify results (Save or Not)
            img_ext = ('.jpg', '.jpeg', '.png')
            if os.path.exists(output_path):
                final_count = len([f for f in os.listdir(output_path) if f.lower().endswith(img_ext)])
                print(f"Verification: [{final_count}] files in output directory.")
            else:
                print(f"Warning!!-Output directory [{output_path}] does not exist for class [{class_name}]")
        
        except Exception as e:
            print(f"Failed to process class: [{class_name}]: {str(e)}")
            continue    

In [9]:
# Target distribution for classes
# Since 'Monkeypox' class has 284 images 
class_targets = {'Chickenpox': 284,
                 'Cowpox': 284,
                 'Healthy': 284,
                 'HFMD': 284,
                 'Measles': 284,     
                }

In [10]:
# Call and run the multi class augmentation funtion
multi_aug_classes(input_dir='F:/MobileNetV2/Dataset/Skin diseases data',
                  output_dir='F:/MobileNetV2/Dataset/augmented images', 
                  class_targets=class_targets,
                  save_format='.jpg', 
                  random_seed=42)

Processing class: [Chickenpox]
Found [75] images in [F:/MobileNetV2/Dataset/Skin diseases data\Chickenpox] path.
Output directory created or verified : [F:/MobileNetV2/Dataset/augmented images\Chickenpox].
Need to generate [209] additional images to balance the dataset.
Generating [2] images per original, with [59] images getting one extra.


Augmenting images:   0%|          | 0/75 [00:00<?, ?it/s]

Augmenting images: 100%|██████████| 75/75 [00:05<00:00, 12.66it/s]


Augmentation complete! Generated [209] new images.
Total images now [284] new images.
Verification: [209] files in output directory.
Processing class: [Cowpox]
Found [66] images in [F:/MobileNetV2/Dataset/Skin diseases data\Cowpox] path.
Output directory created or verified : [F:/MobileNetV2/Dataset/augmented images\Cowpox].
Need to generate [218] additional images to balance the dataset.
Generating [3] images per original, with [20] images getting one extra.


Augmenting images: 100%|██████████| 66/66 [00:06<00:00, 10.38it/s]


...

Augmentation complete! Generated [170] new images.
Total images now [284] new images.
Verification: [170] files in output directory.
Processing class: [HFMD]
Found [161] images in [F:/MobileNetV2/Dataset/Skin diseases data\HFMD] path.
Output directory created or verified : [F:/MobileNetV2/Dataset/augmented images\HFMD].
Need to generate [123] additional images to balance the dataset.
Generating [0] images per original, with [123] images getting one extra.


Augmenting images: 100%|██████████| 161/161 [00:16<00:00,  9.49it/s]


Augmentation complete! Generated [123] new images.
Total images now [284] new images.
Verification: [123] files in output directory.
Processing class: [Measles]
Found [55] images in [F:/MobileNetV2/Dataset/Skin diseases data\Measles] path.
Output directory created or verified : [F:/MobileNetV2/Dataset/augmented images\Measles].
Need to generate [229] additional images to balance the dataset.
Generating [4] images per original, with [9] images getting one extra.


Augmenting images: 100%|██████████| 55/55 [00:05<00:00,  9.25it/s]

Augmentation complete! Generated [229] new images.
Total images now [284] new images.
Verification: [229] files in output directory.


In [12]:
# Analyze dataset after agumentation 
analyze_dataset(dir="F:/MobileNetV2/Dataset/Skin diseases data")

Total Images : 1704
[Chickenpox] : 284 Images, [16.7%]
[Cowpox] : 284 Images, [16.7%]
[HFMD] : 284 Images, [16.7%]
[Healthy] : 284 Images, [16.7%]
[Measles] : 284 Images, [16.7%]
[Monkeypox] : 284 Images, [16.7%]


{'Chickenpox': 284,
 'Cowpox': 284,
 'Healthy': 284,
 'HFMD': 284,
 'Measles': 284,
 'Monkeypox': 284}

# **Split Raw Image Data Into Train & Vaild :**

In [14]:
def split_dataset(input_dir, output_dir, train_ratio=0.8, seed=42):
    if not input_dir or not os.path.exists(input_dir):
        print("Input directory can not found.")
        return False
    else:
        print("Input directory found sucessfully and continue the process.")
    
    random.seed(seed)
    input_dir=Path(input_dir)
    output_dir=Path(output_dir)
    total_files=0
    processed_files=0
    
    # Create the output directory
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Iterate through class directories
    for cls_dir in input_dir.iterdir():
        if cls_dir.is_dir():
            img_ext = ['*.jpg', '*.jpeg', '*.png']
            imgs=[]
            
            for ext in img_ext:
                imgs.extend(cls_dir.glob(ext))
                imgs.extend(cls_dir.glob(ext.upper())) # For upper case ext.
            
            if not imgs:
                print(f"Warning!!-No image files found in [{cls_dir.name}].")
                continue 
            
            print(f"Processing class [{cls_dir.name}] with [{len(imgs)}] images.")
            total_files += len(imgs)

            # Shuffle images randomly
            random.shuffle(imgs)
            
            # calculate split index
            split_idx=int(len(imgs) * train_ratio)
            train_imgs=imgs[:split_idx]
            val_imgs=imgs[split_idx:]
            
            print(f"Train - [{len(train_imgs)}] , Validation - [{len(val_imgs)}].")
            
            # Copy files to output directories
            for split, split_imgs in [("train", train_imgs), ("val", val_imgs)]:
                target_dir= output_dir / split / cls_dir.name
                target_dir.mkdir(parents=True, exist_ok=True)
                
                for img_path in split_imgs:
                    try:
                        shutil.copy(img_path, target_dir / img_path.name)
                        processed_files +=1
                    except Exception as e:
                        print(f"Error occurred during copying [{img_path.name}]: {e}")
                        
    print(f"Dataset split completed. Processed [{processed_files}/{total_files}] files.")
    
    return processed_files == total_files
                        

In [15]:
split_dataset(input_dir="F:/MobileNetV2/Dataset/Skin diseases data",
              output_dir="F:/MobileNetV2/Dataset/Skin diseases splited Dataset",
              train_ratio=0.8, 
              seed=42)

Input directory found sucessfully and continue the process.
Processing class [Chickenpox] with [568] images.
Train - [454] , Validation - [114].
Processing class [Cowpox] with [568] images.
Train - [454] , Validation - [114].
Processing class [Healthy] with [568] images.
Train - [454] , Validation - [114].
Processing class [HFMD] with [568] images.
Train - [454] , Validation - [114].
Processing class [Measles] with [568] images.
Train - [454] , Validation - [114].
Processing class [Monkeypox] with [568] images.
Train - [454] , Validation - [114].
Dataset split completed. Processed [3408/3408] files.


True